# Char-RNN
Reading Ref: [http://karpathy.github.io/2015/05/21/rnn-effectiveness/](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

# Text Generation


Download the Shakespear dataset from [Andrej's Blog](https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt)

Approach (broader view):
- load the dataset
- convert categorical values into some numerical representation; we'll create mapping of char to int
- setup the sequence logic
```
if the sequence length is 4,
HELL->O
WORL->D
```
- set up the LSTM architecture
- train it
- see the results


In [1]:
# import necessary libraries
import sys
import numpy as np
import pandas as pd
import sys
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\z003w00f\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\z003w00f\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np

In [2]:
# loading the dataset
filename="shakespear.txt"
raw_text= open(filename, 'r', encoding='utf-8').read()
raw_text= raw_text.lower()

In [3]:
# create mapping of all unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [4]:
print(chars)

['\n', ' ', '!', "'", ',', '-', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
n_chars = len(raw_text)
n_vocabs = len(chars)
print("Total characters: ", n_chars)
print("Total vocab: ", n_vocabs)

Total characters:  99993
Total vocab:  36


In [6]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length , 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append([char_to_int[seq_out]])

n_patterns = len(dataX)
print("Total patterns: ", n_patterns)

Total patterns:  99893


since LSTMs accept values in the form (no_of_sampels, time_steps, no_of_features), therefore<br>
reshape dataX to this form

In [7]:
# reshape dataX
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X/float(n_vocabs)

# one hot encoding using np_utils
y = np_utils.to_categorical(dataY)

In [8]:
X.shape

(99893, 100, 1)

In [9]:
y.shape

(99893, 36)

In [10]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(LSTM(256))
model.add(Dropout(rate=0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [16]:
# define the checkpoints and callbacks
filepath="SavedModels/weights-imporvement-{epoch: 02d}-{loss: .4f}-from-class.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [17]:
# Generating Text from pretrained/post training your LSTM

filename = "SavedModels/weights-imporvement- 50- 1.3274-from-class.hdf5" # add the name of your best trained saved model's name here
model.load_weights(filename)
#model.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
# training the LSTM

model.fit(X, y, epochs=100, batch_size=64, callbacks=callbacks_list,initial_epoch=50) # do try out at diff epoch and batch sizes

Epoch 51/100
99893/99893 [==============================] - 1137s 11ms/step - loss: 1.3208

Epoch 00051: loss improved from inf to 1.32079, saving model to SavedModels/weights-imporvement- 51- 1.3208-from-class.hdf5
Epoch 52/100
99893/99893 [==============================] - 1133s 11ms/step - loss: 1.3189

Epoch 00052: loss improved from 1.32079 to 1.31890, saving model to SavedModels/weights-imporvement- 52- 1.3189-from-class.hdf5
Epoch 53/100
99893/99893 [==============================] - 1129s 11ms/step - loss: 1.3109

Epoch 00053: loss improved from 1.31890 to 1.31091, saving model to SavedModels/weights-imporvement- 53- 1.3109-from-class.hdf5
Epoch 54/100
99893/99893 [==============================] - 1128s 11ms/step - loss: 1.3098

Epoch 00054: loss improved from 1.31091 to 1.30983, saving model to SavedModels/weights-imporvement- 54- 1.3098-from-class.hdf5
Epoch 55/100
99893/99893 [==============================] - 1131s 11ms/step - loss: 1.3061

Epoch 00055: loss improved from 

99893/99893 [==============================] - 1154s 12ms/step - loss: 1.2199

Epoch 00094: loss did not improve from 1.21783
Epoch 95/100
99893/99893 [==============================] - 1157s 12ms/step - loss: 1.2211

Epoch 00095: loss did not improve from 1.21783
Epoch 96/100
99893/99893 [==============================] - 1147s 11ms/step - loss: 1.2196

Epoch 00096: loss did not improve from 1.21783
Epoch 97/100
99893/99893 [==============================] - 1149s 11ms/step - loss: 1.2250

Epoch 00097: loss did not improve from 1.21783
Epoch 98/100
99893/99893 [==============================] - 1149s 12ms/step - loss: 1.2281

Epoch 00098: loss did not improve from 1.21783
Epoch 99/100
99893/99893 [==============================] - 1155s 12ms/step - loss: 1.2316

Epoch 00099: loss did not improve from 1.21783
Epoch 100/100
99893/99893 [==============================] - 1161s 12ms/step - loss: 1.2255

Epoch 00100: loss did not improve from 1.21783


In [20]:
# set up a random seed for starting
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]

print("INPUT SEED:")
print("\"", ''.join([int_to_char[val] for val in pattern]), "\"")
print()
# generate characters from the generated output of LSTM
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/float(n_vocabs)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1: len(pattern)]
print("\nTHE END.")

INPUT SEED:
" aff:
what, say you thou art any happiness?

first murderer:
there's some but hot.

orlando:
basis i  "

lay see the fire that same that i will sea the will be sale;

second lord:
therefore, i lied ho suear like a song tiat the day that would poent be to my father's servers.

antonio:
a withug to the martiage,
mord a song that i will prove be well and light to the puhen eriends; the heavens and to me,
the way to be so.

second servingman:
now, he is may not prove her head and single shale as the pueen and to me,
the which in the fortunes to the torenes;
there are the heavens and conpent the sight of nur offecor.

arutus:
mo more than the will grom the but to cear as he was cesices him in thy food court,
to see the duke of suranger, and in the pueen of the part.

crutus:
we will as my poenet, and in the pueen of the court,

tecond lord:
therefore, i lied ho saint and the surengt of lind of your frace and the single sraite,
they have here iim to my father and in the foreht c